# Import Libraries

In [1]:
import pandas as pd

# For Association Rules Learning & Apriori 
from mlxtend.frequent_patterns import apriori, association_rules

# Setting Configurations:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Import Warnings:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

# Raw Code

## Loading and pre-processing of data

In [18]:
df=pd.read_csv('dataset/Groceries_dataset.csv')
print(df.shape)
df.isna().sum()

(38765, 3)


Member_number      0
Date               0
itemDescription    0
dtype: int64

In [19]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [20]:
df['Member_number'].nunique()

3898

In [21]:
df['itemDescription'].nunique()

167

In [22]:
df.insert(3,'Quantity',1)

In [23]:
df

,Member_number,Date,itemDescription,Quantity
0,1808,21-07-2015,tropical fruit,1
1,2552,05-01-2015,whole milk,1
2,2300,19-09-2015,pip fruit,1
3,1187,12-12-2015,other vegetables,1
4,3037,01-02-2015,whole milk,1
...,...,...,...,...
38760,4471,08-10-2014,sliced cheese,1
38761,2022,23-02-2014,candy,1
38762,1097,16-04-2014,cake bar,1
38763,1510,03-12-2014,fruit/vegetable juice,1


## Algorithm

In [24]:
matrix=df.groupby(['Member_number', "itemDescription"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
matrix

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Member_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1001,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
1002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [25]:
frequent_itemsets = apriori(matrix, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.015,(Instant food products)
1,0.079,(UHT-milk)
2,0.031,(baking powder)
3,0.120,(beef)
4,0.080,(berries)
...,...,...
3011,0.011,"(whipped/sour cream, soda, whole milk, yogurt)"
3012,0.011,"(other vegetables, yogurt, rolls/buns, bottled..."
3013,0.014,"(sausage, other vegetables, yogurt, rolls/buns..."
3014,0.010,"(other vegetables, yogurt, shopping bags, roll..."


In [26]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.sort_values("support", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1868,(other vegetables),(whole milk),0.377,0.458,0.191,0.508,1.109,0.019,1.102
1869,(whole milk),(other vegetables),0.458,0.377,0.191,0.418,1.109,0.019,1.071
2048,(whole milk),(rolls/buns),0.458,0.350,0.179,0.390,1.114,0.018,1.066
2049,(rolls/buns),(whole milk),0.350,0.458,0.179,0.511,1.114,0.018,1.107
2187,(whole milk),(soda),0.458,0.313,0.151,0.330,1.052,0.007,1.024


In [27]:
sorted_rules = rules.sort_values("lift", ascending=False)
sorted_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15412,"(yogurt, rolls/buns)","(sausage, other vegetables, whole milk)",0.111,0.050,0.014,0.122,2.429,0.008,1.082
15397,"(sausage, other vegetables, whole milk)","(yogurt, rolls/buns)",0.050,0.111,0.014,0.270,2.429,0.008,1.218
15401,"(other vegetables, yogurt, rolls/buns)","(sausage, whole milk)",0.052,0.107,0.014,0.260,2.429,0.008,1.206
15408,"(sausage, whole milk)","(other vegetables, yogurt, rolls/buns)",0.107,0.052,0.014,0.127,2.429,0.008,1.086
13016,"(yogurt, curd)","(sausage, whole milk)",0.040,0.107,0.010,0.248,2.322,0.006,1.188
...,...,...,...,...,...,...,...,...,...
8435,(long life bakery product),"(other vegetables, whole milk)",0.065,0.191,0.011,0.169,0.881,-0.001,0.973
8846,"(sausage, other vegetables)",(newspapers),0.093,0.140,0.011,0.122,0.869,-0.002,0.979
8851,(newspapers),"(sausage, other vegetables)",0.140,0.093,0.011,0.081,0.869,-0.002,0.987
885,(cream cheese ),(citrus fruit),0.089,0.185,0.014,0.159,0.860,-0.002,0.969


## Recommendation using apriori algorithm

In [28]:
product_id = 'meat'
recommendation_list = []


for idx, product in enumerate(sorted_rules["antecedents"]):
    #Since the antecedents is a tuple, let's convert it to a list and search within the list:
    for j in list(product):
        if j == product_id:
            #Whatever the index (idx) of this integer value we have captured, we will search for it in the consequent, let's suggest the first product [0] for the rows we find
            recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
            recommendation_list = list( dict.fromkeys(recommendation_list) )

In [29]:
list_top5 = recommendation_list[0:5]
list_top5

['domestic eggs',
 'whole milk',
 'other vegetables',
 'tropical fruit',
 'root vegetables']

# Python Function

In [7]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


def generate_association_rules(data_path, target_product, min_support=0.01, min_threshold=0.01, top_n=5):
    """
    Generate association rules for a given target product based on the Apriori algorithm.

    Parameters:
    - data_path (str): Path to the CSV file containing the dataset.
    - target_product (str): The target product for which recommendations will be generated.
    - min_support (float): Minimum support threshold for Apriori algorithm. Default is 0.01.
    - min_threshold (float): Minimum threshold for association rules. Default is 0.01.
    - top_n (int): Number of top recommendations to return. Default is 5.

    Returns:
    - list: Top N recommended products for the given target product.
    """
    
    # Read the dataset
    df=pd.read_csv(path)
    df.insert(3,'Quantity',1)

    # Create a binary matrix indicating whether each product was purchased by a member
    matrix = df.groupby(['Member_number', 'itemDescription'])['Quantity'].sum().unstack().fillna(0). \
        applymap(lambda x: 1 if x > 0 else 0)

    # Generate frequent itemsets using Apriori algorithm
    frequent_itemsets = apriori(matrix, min_support=min_support, use_colnames=True)

    # Generate association rules
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=min_threshold)

    # Sort the rules by lift in descending order
    sorted_rules = rules.sort_values("lift", ascending=False)

    # Extract top N recommendations for the target product
    recommendation_list = []
    for _, product in enumerate(sorted_rules["antecedents"]):
        if target_product in list(product):
            recommendation_list.append(list(sorted_rules.iloc[_]["consequents"])[0])
            recommendation_list = list(dict.fromkeys(recommendation_list))

    return recommendation_list[:top_n]


In [8]:
# Example Usage:
target_product_id = 'meat'
path = 'dataset/Groceries_dataset.csv'
top_recommendations = generate_association_rules(path, target_product_id)
print(top_recommendations)

['whole milk', 'domestic eggs', 'root vegetables', 'tropical fruit', 'canned beer']
